In [1]:
using Pkg
Pkg.activate("MyProject")

using JuMP
const jmp = JuMP

using GLPK

  Activating project at `~/GitHub/StabTheory/MyProject`


In [2]:
include("libs/cnc.jl"); include("libs/stab.jl"); include("libs/symplectic.jl"); include("libs/initializer.jl")

In [159]:
using JuMP
const jmp = JuMP

using GLPK

function initial_lp(V,R)
    d = size(R)[1]; N = size(R)[2]

    # define model:
    model = jmp.Model(GLPK.Optimizer)

    # define variables: non-negative:
    @variable(model,x[1:N] >= 0)

    # define constraints:
    @constraint(model, cons, R * x .== V)

    # define objective function: We only want to find a solution:
    @objective(model, Min, 0)

    # solve model
    optimize!(model)

    if is_solved_and_feasible(model) == true
        return model,x
    else
        # define model:
        model = jmp.Model(GLPK.Optimizer)

        # define variables: non-negative:
        @variable(model,x[1:N])

        # define constraints:
        @constraint(model, cons, R * x .== V)

        # define objective function: We only want to find a solution:
        @objective(model, Min, 0)

        # solve model
        optimize!(model)

        return model,x
    end
end

initial_lp (generic function with 1 method)

In [140]:
function approximate_input(V,e)
    W = [Float64(round(BigFloat(v);digits = e)) for v in V]
    return W
end

approximate_input (generic function with 1 method)

In [200]:
arry = Array{Float64}(undef, 2, 0)
hcat(arry,Vector([2,1]))

2×1 Matrix{Float64}:
 2.0
 1.0

In [214]:
function tight_vertices(V,A,R)
    """
    V: vector dx1
    A: matrix Mxd
    R: matrix dxN
    """
    Z = findall(x->x==0,A*V); D = length(Z)
    if D == 0
        return R
    else
        ARZ = A[Z,:]*R
        RZ = Array{Rational{Int64}}(undef,size(R)[1],0)
        for j in 1:size(R)[2]
            if ARZ[:,j] == zeros(D); RZ = hcat(RZ,R[:,j]); end;
        end
        return RZ
    end
end

tight_vertices (generic function with 1 method)

# $1$-qubit magic state:

Using stabilizer states as a quasiprobability model for simplicity.

In [4]:
R1 = transpose(stabilizer_states(1))

4×6 transpose(::Matrix{Int64}) with eltype Int64:
 1   1  1   1  1   1
 1  -1  0   0  0   0
 0   0  0   0  1  -1
 0   0  1  -1  0   0

In [5]:
T1 = [1,1/sqrt(2),0,1/sqrt(2)];

In [253]:
model, x = initial_lp(T1,R1)

(A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: cons, x, VariableRef[x[1], x[2], x[3], x[4], x[5], x[6]])

In [184]:
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Objective bound    : -Inf
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.69277e-05


In [185]:
# Print negative values
for i in 1:length(value.(x))
    v = value.(x)[i]
    if v < 0; println("Stabilizer state: ",i,", Coefficient: ",v); end;
end

Stabilizer state: 4, Coefficient: -0.20710678118654746


In [11]:
InitT1 = Initializer(1,T1,R1)

Initializer([1.0, 0.7071067811865475, 0.0, 0.7071067811865475], false, MaximalCnc[CNC:
Maximal CNC set:
- Isotropic generators:
  ["X"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  I: 1
  X: 1
, CNC:
Maximal CNC set:
- Isotropic generators:
  ["Z"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  I: 1
  Z: 1
], Dict{MaximalCnc, Float64}(CNC:
Maximal CNC set:
- Isotropic generators:
  ["X"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  I: 1
  X: 1
 => 0.7071067811865475, CNC:
Maximal CNC set:
- Isotropic generators:
  ["Z"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  I: 1
  Z: 1
 => 0.29289321881345254))

# $2$-qubit Magic state:

In [12]:
A2 = (stabilizer_states(2)); R2 = transpose(A2);

In [13]:
T2 = [1,1/sqrt(2),0,1/sqrt(2),1/sqrt(2),1/2,0,1/2,0,0,0,0,1/sqrt(2),1/2,0,1/2]

16-element Vector{Float64}:
 1.0
 0.7071067811865475
 0.0
 0.7071067811865475
 0.7071067811865475
 0.5
 0.0
 0.5
 0.0
 0.0
 0.0
 0.0
 0.7071067811865475
 0.5
 0.0
 0.5

In [14]:
Q2 = approximate_input(T2,2)

16-element Vector{Float64}:
 1.0
 0.71
 0.0
 0.71
 0.71
 0.5
 0.0
 0.5
 0.0
 0.0
 0.0
 0.0
 0.71
 0.5
 0.0
 0.5

In [228]:
model, x = initial_lp(Q2,R2)

(A JuMP Model
Minimization problem with:
Variables: 60
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 16 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: cons, x, VariableRef[x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10]  …  x[51], x[52], x[53], x[54], x[55], x[56], x[57], x[58], x[59], x[60]])

In [232]:
# Print negative values
for i in 1:length(value.(x))
    v = value.(x)[i]
    if v != 0; println("Stabilizer state: ",i,", Coefficient: ",v); end;
end

Stabilizer state: 1, Coefficient: 0.3499999999999999
Stabilizer state: 9, Coefficient: 0.35000000000000003
Stabilizer state: 13, Coefficient: 0.35000000000000003
Stabilizer state: 25, Coefficient: 0.32499999999999996
Stabilizer state: 28, Coefficient: -0.024999999999999967
Stabilizer state: 35, Coefficient: -0.024999999999999967
Stabilizer state: 36, Coefficient: -0.175
Stabilizer state: 48, Coefficient: -0.15000000000000002


In [230]:
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Objective bound    : -Inf
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 4.50611e-05


In [16]:
InitT2 = Initializer(2,Q2,R2)

Initializer([1.0, 0.71, 0.0, 0.71, 0.71, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.71, 0.5, 0.0, 0.5], false, MaximalCnc[CNC:
Maximal CNC set:
- Isotropic generators:
  ["IX", "XX"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IX: 1
  II: 1
  XX: 1
  XI: 1
, CNC:
Maximal CNC set:
- Isotropic generators:
  ["IZ", "XZ"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IZ: 1
  II: 1
  XZ: 1
  XI: 1
, CNC:
Maximal CNC set:
- Isotropic generators:
  ["IX", "ZX"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IX: 1
  II: 1
  ZX: 1
  ZI: 1
, CNC:
Maximal CNC set:
- Isotropic generators:
  ["IZ", "ZZ"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IZ: 1
  II: 1
  ZZ: 1
  ZI: 1
], Dict{MaximalCnc, Float64}(CNC:
Maximal CNC set:
- Isotropic generators:
  ["IZ", "XZ"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IZ: 1
  II: 1
  XZ: 1
  XI: 1
 => 0.20999999999999996, CNC:
Maximal CNC set:
- Isotropic generators:
  ["IZ", "ZZ"]
- Anticommuting Pa

# Maximally mixed state:

In [21]:
MIX = Vector{Float64}(vcat([1],[0 for i in 1:15]))

16-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [22]:
InitMIX = Initializer(2,MIX,R2)

Initializer([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], true, MaximalCnc[CNC:
Maximal CNC set:
- Isotropic generators:
  ["IX", "XX"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IX: 1
  II: 1
  XX: 1
  XI: 1
, CNC:
Maximal CNC set:
- Isotropic generators:
  ["IX", "XX"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IX: -1
  II: 1
  XX: -1
  XI: 1
, CNC:
Maximal CNC set:
- Isotropic generators:
  ["IZ", "XZ"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IZ: 1
  II: 1
  XZ: -1
  XI: -1
, CNC:
Maximal CNC set:
- Isotropic generators:
  ["IZ", "XZ"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IZ: -1
  II: 1
  XZ: 1
  XI: -1
], Dict{MaximalCnc, Float64}(CNC:
Maximal CNC set:
- Isotropic generators:
  ["IX", "XX"]
- Anticommuting Paulis:
  String[]

- Value assignment:
  IX: -1
  II: 1
  XX: -1
  XI: 1
 => 0.25, CNC:
Maximal CNC set:
- Isotropic generators:
  ["IZ", "XZ"]
- Anticommuting Paulis:
  Stri

In [23]:
InitMIX.InitDist

Dict{MaximalCnc, Float64} with 4 entries:
  CNC:…                                                                   => 0.25
  CNC:…                                                                   => 0.25
  CNC:…                                                                   => 0.25
  CNC:…                                                                   => 0.25